# Multivariable Calculus & ODEs

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** 57-84
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
# Standard scientific stack
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from sympy import (symbols, diff, integrate, sin, cos, exp, sqrt, pi,
                   Function, Eq, dsolve, classify_ode, simplify, Matrix,
                   latex, Rational)
from scipy.integrate import solve_ivp
from scipy.integrate import quad

%matplotlib inline

# Publication-quality defaults
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 15,
    'lines.linewidth': 2,
    'legend.fontsize': 11,
    'figure.dpi': 100,
})

sp.init_printing(use_unicode=True)
print('Libraries loaded successfully.')

## 1. Multivariable Functions and 3D Visualization

A function of two variables $f(x, y)$ maps points in $\mathbb{R}^2$ to
$\mathbb{R}$. Its graph is a **surface** in 3D space.

### Partial derivatives

$$\frac{\partial f}{\partial x} = \lim_{h\to 0}\frac{f(x+h,y)-f(x,y)}{h}, \qquad
\frac{\partial f}{\partial y} = \lim_{h\to 0}\frac{f(x,y+h)-f(x,y)}{h}$$

### The gradient

$$\nabla f = \left(\frac{\partial f}{\partial x},\, \frac{\partial f}{\partial y}\right)$$

points in the direction of steepest ascent and has magnitude equal to the
maximum rate of change.

In [ ]:
# 3D surface plots for multivariable functions
x_vals = np.linspace(-3, 3, 100)
y_vals = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x_vals, y_vals)

# Four interesting surfaces
surfaces = {
    r'$f = x^2 + y^2$ (Paraboloid)':       X**2 + Y**2,
    r'$f = \sin(x)\cos(y)$ (Saddle-like)':  np.sin(X) * np.cos(Y),
    r'$f = e^{-(x^2+y^2)}$ (Gaussian)':     np.exp(-(X**2 + Y**2)),
    r'$f = xy\,e^{-(x^2+y^2)/2}$ (Monkey saddle)': X * Y * np.exp(-(X**2 + Y**2)/2),
}

fig = plt.figure(figsize=(14, 11))
for i, (title, Z) in enumerate(surfaces.items(), 1):
    ax = fig.add_subplot(2, 2, i, projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.85,
                           edgecolor='none')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('f(x,y)')
    ax.set_title(title, fontsize=12, pad=10)
    ax.view_init(elev=30, azim=135)

plt.suptitle('Surfaces in 3D', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

## 2. Gradient Visualization with Quiver Plots

The gradient field $\nabla f$ can be visualized as arrows on the $xy$-plane,
overlaid on a contour plot of $f$. The arrows point uphill, and their length
encodes the steepness.

For the **paraboloid** $f(x,y) = x^2 + y^2$:

$$\nabla f = (2x, \, 2y)$$

The gradient points radially outward from the origin -- everywhere perpendicular
to the circular level curves.

In [ ]:
# Gradient visualization with quiver plots on contour backgrounds
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Function 1: Paraboloid
x_grid = np.linspace(-3, 3, 20)
y_grid = np.linspace(-3, 3, 20)
Xq, Yq = np.meshgrid(x_grid, y_grid)

# Fine grid for contours
x_fine = np.linspace(-3, 3, 200)
y_fine = np.linspace(-3, 3, 200)
Xf, Yf = np.meshgrid(x_fine, y_fine)

# (a) Paraboloid: f = x^2 + y^2
Z1 = Xf**2 + Yf**2
U1, V1 = 2*Xq, 2*Yq  # Gradient components

axes[0].contour(Xf, Yf, Z1, levels=15, cmap='coolwarm', alpha=0.6)
axes[0].quiver(Xq, Yq, U1, V1, color='black', alpha=0.7, scale=60)
axes[0].set_title(r'$\nabla(x^2 + y^2) = (2x, 2y)$')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_aspect('equal')

# (b) Saddle: f = x^2 - y^2
Z2 = Xf**2 - Yf**2
U2, V2 = 2*Xq, -2*Yq

axes[1].contour(Xf, Yf, Z2, levels=15, cmap='coolwarm', alpha=0.6)
axes[1].quiver(Xq, Yq, U2, V2, color='black', alpha=0.7, scale=60)
axes[1].set_title(r'$\nabla(x^2 - y^2) = (2x, -2y)$')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_aspect('equal')

# (c) Gaussian: f = exp(-(x^2+y^2))
Z3 = np.exp(-(Xf**2 + Yf**2))
U3 = -2*Xq * np.exp(-(Xq**2 + Yq**2))
V3 = -2*Yq * np.exp(-(Xq**2 + Yq**2))

axes[2].contour(Xf, Yf, Z3, levels=15, cmap='coolwarm', alpha=0.6)
axes[2].quiver(Xq, Yq, U3, V3, color='black', alpha=0.7, scale=5)
axes[2].set_title(r'$\nabla e^{-(x^2+y^2)}$')
axes[2].set_xlabel('x')
axes[2].set_ylabel('y')
axes[2].set_aspect('equal')

plt.suptitle('Gradient Fields (arrows) over Contour Plots (level curves)',
             fontsize=14, y=1.03)
plt.tight_layout()
plt.show()
print('Gradient arrows are perpendicular to contour lines -- they point uphill.')

## 3. Line Integrals and Green's Theorem

### Line integral of a vector field

$$\oint_C \mathbf{F}\cdot d\mathbf{r} = \int_a^b \mathbf{F}(\mathbf{r}(t))\cdot\mathbf{r}'(t)\,dt$$

### Green's Theorem

For a simply connected region $D$ bounded by a positively oriented curve $C$:

$$\oint_C (P\,dx + Q\,dy) = \iint_D \left(\frac{\partial Q}{\partial x} - \frac{\partial P}{\partial y}\right) dA$$

This relates a line integral around a closed curve to a double integral over the enclosed region.
Green's theorem is a precursor to **Stokes' theorem**, which is fundamental in
electromagnetism and gauge theories in quantum field theory.

In [ ]:
# Green's Theorem verification
# Vector field F = (-y, x) around the unit circle
# Line integral: integral of (-y dx + x dy) around C
# Green's: double integral of (dQ/dx - dP/dy) dA = (1 - (-1)) dA = 2 * Area

x, y, t = symbols('x y t', real=True)

# Parametrize the unit circle: r(t) = (cos t, sin t), t in [0, 2pi]
x_t = cos(t)
y_t = sin(t)
dx_dt = diff(x_t, t)  # -sin(t)
dy_dt = diff(y_t, t)  # cos(t)

# P = -y, Q = x
P = -y_t  # -sin(t)
Q = x_t   # cos(t)

# Line integral: integral of (P*dx/dt + Q*dy/dt) dt from 0 to 2pi
integrand = P * dx_dt + Q * dy_dt
line_integral = integrate(integrand, (t, 0, 2*pi))

# Green's theorem: dQ/dx - dP/dy = 1 - (-1) = 2
# Double integral over unit disk: 2 * pi * r^2 = 2*pi
greens_result = 2 * pi  # 2 * area of unit circle

print("Green's Theorem Verification")
print('=' * 50)
print(f'  Vector field: F = (-y, x)')
print(f'  Curve: unit circle (counterclockwise)')
print(f'')
print(f'  Line integral:     {line_integral}')
print(f"  Green's (2*Area):  {greens_result}")
print(f'  Match: {simplify(line_integral - greens_result) == 0}')

# Visualization: the vector field and the unit circle
theta = np.linspace(0, 2*np.pi, 100)
circle_x = np.cos(theta)
circle_y = np.sin(theta)

# Vector field on a grid
xg = np.linspace(-1.5, 1.5, 12)
yg = np.linspace(-1.5, 1.5, 12)
Xg, Yg = np.meshgrid(xg, yg)
Ug = -Yg   # P = -y
Vg = Xg    # Q = x

fig, ax = plt.subplots(figsize=(7, 7))
ax.quiver(Xg, Yg, Ug, Vg, color='steelblue', alpha=0.7, scale=20)
ax.plot(circle_x, circle_y, 'r-', lw=2.5, label='Unit circle C')

# Add direction arrow on circle
arrow_idx = 10
ax.annotate('', xy=(circle_x[arrow_idx+3], circle_y[arrow_idx+3]),
            xytext=(circle_x[arrow_idx], circle_y[arrow_idx]),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title(r'$\mathbf{F} = (-y, x)$ and unit circle $C$'
             f'\nLine integral = Green\'s = {float(line_integral):.4f}')
ax.set_aspect('equal')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Ordinary Differential Equations (ODEs)

An ODE relates a function to its derivatives. The general first-order ODE:

$$\frac{dy}{dx} = f(x, y)$$

### Classification:
- **Separable:** $\frac{dy}{dx} = g(x)h(y)$
- **Linear first-order:** $\frac{dy}{dx} + P(x)y = Q(x)$
- **Exact:** $M(x,y)dx + N(x,y)dy = 0$ where $\frac{\partial M}{\partial y} = \frac{\partial N}{\partial x}$

### Second-order linear ODEs with constant coefficients:

$$ay'' + by' + cy = f(x)$$

The characteristic equation $ar^2 + br + c = 0$ determines the solution type.

In [ ]:
# Analytical ODE solving with SymPy's dsolve
x = symbols('x')
y = Function('y')

print('ANALYTICAL ODE SOLUTIONS (SymPy dsolve)')
print('=' * 65)

odes = [
    # (equation, description)
    (Eq(y(x).diff(x), 2*x),
     "y' = 2x  (direct integration)"),
    (Eq(y(x).diff(x), -y(x)),
     "y' = -y  (exponential decay)"),
    (Eq(y(x).diff(x), y(x)*(1 - y(x))),
     "y' = y(1-y)  (logistic equation)"),
    (Eq(y(x).diff(x, 2) + y(x), 0),
     "y'' + y = 0  (simple harmonic oscillator)"),
    (Eq(y(x).diff(x, 2) + 2*y(x).diff(x) + y(x), 0),
     "y'' + 2y' + y = 0  (critically damped)"),
    (Eq(y(x).diff(x, 2) + y(x).diff(x) + 5*y(x), 0),
     "y'' + y' + 5y = 0  (underdamped)"),
]

for eq, desc in odes:
    sol = dsolve(eq, y(x))
    ode_class = classify_ode(eq, y(x))
    print(f'\n  {desc}')
    print(f'    Classification: {ode_class[0]}')
    print(f'    Solution: {sol}')

In [ ]:
# Numerical ODE solving with scipy solve_ivp
# Compare with analytical solutions

# Example: damped harmonic oscillator y'' + 0.5*y' + 4*y = 0
# Rewrite as system: y1 = y, y2 = y'
# y1' = y2, y2' = -0.5*y2 - 4*y1

def damped_oscillator(t, state, gamma=0.5, omega2=4.0):
    '''Damped harmonic oscillator: y\'\'+ gamma*y\' + omega^2*y = 0.'''
    y, v = state
    return [v, -gamma * v - omega2 * y]

# Initial conditions: y(0) = 1, y'(0) = 0
y0 = [1.0, 0.0]
t_span = (0, 15)
t_eval = np.linspace(*t_span, 500)

# Solve for different damping levels
damping_levels = {
    'Underdamped (gamma=0.3)':  0.3,
    'Critically damped (gamma=4.0)': 4.0,
    'Overdamped (gamma=5.0)':  5.0,
}

fig, ax = plt.subplots(figsize=(11, 6))

for label, gamma in damping_levels.items():
    sol = solve_ivp(damped_oscillator, t_span, y0, t_eval=t_eval,
                    args=(gamma, 4.0), method='RK45', rtol=1e-10)
    ax.plot(sol.t, sol.y[0], lw=2, label=label)

ax.axhline(0, color='gray', ls='--', alpha=0.4)
ax.set_xlabel('Time t')
ax.set_ylabel('y(t)')
ax.set_title(r"Damped Harmonic Oscillator: $y'' + \gamma y' + 4y = 0$, $y(0)=1$")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print('Underdamped: oscillations decay. Critical: fastest decay without overshoot.')
print('Overdamped: slow return to equilibrium without oscillation.')

## 5. Phase Portraits for 2D Systems

A 2D autonomous system

$$\frac{dx}{dt} = f(x, y), \qquad \frac{dy}{dt} = g(x, y)$$

can be visualized via a **phase portrait**: a vector field in the $(x, y)$
plane with trajectories showing how the system evolves from different initial conditions.

### Classification of equilibria (linear systems $\dot{\mathbf{x}} = A\mathbf{x}$):

| Eigenvalues | Type |
|---|---|
| Real, same sign | Node (stable if negative) |
| Real, opposite sign | Saddle point |
| Complex, nonzero real part | Spiral (stable if Re < 0) |
| Pure imaginary | Center (periodic orbits) |

In [ ]:
# Phase portraits for three types of 2D linear systems
def phase_portrait(A, ax, title, xlim=(-3, 3), ylim=(-3, 3), n_traj=12):
    '''Draw phase portrait for dx/dt = Ax.'''
    # Vector field
    x_range = np.linspace(*xlim, 20)
    y_range = np.linspace(*ylim, 20)
    X, Y = np.meshgrid(x_range, y_range)
    U = A[0, 0]*X + A[0, 1]*Y
    V = A[1, 0]*X + A[1, 1]*Y
    speed = np.sqrt(U**2 + V**2)
    speed[speed == 0] = 1  # Avoid division by zero

    ax.streamplot(X, Y, U, V, color=speed, cmap='coolwarm',
                  density=1.5, linewidth=0.8, arrowsize=1.2)

    # Add some trajectories from specific initial conditions
    theta_vals = np.linspace(0, 2*np.pi, n_traj, endpoint=False)
    for theta in theta_vals:
        ic = [2*np.cos(theta), 2*np.sin(theta)]
        sol = solve_ivp(lambda t, s: A @ s, [0, 10], ic,
                        t_eval=np.linspace(0, 10, 500), method='RK45')
        ax.plot(sol.y[0], sol.y[1], 'k-', alpha=0.3, lw=0.5)

    # Eigenvalues
    eigvals = np.linalg.eigvals(A)
    eig_str = ', '.join([f'{e:.2f}' for e in eigvals])

    ax.plot(0, 0, 'ko', markersize=6)
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'{title}\n$\\lambda$ = {eig_str}')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.2)

fig, axes = plt.subplots(1, 3, figsize=(17, 5.5))

# Stable node
A1 = np.array([[-2, 0], [0, -1]])
phase_portrait(A1, axes[0], 'Stable Node')

# Saddle point
A2 = np.array([[1, 0], [0, -2]])
phase_portrait(A2, axes[1], 'Saddle Point')

# Center (pure imaginary eigenvalues)
A3 = np.array([[0, 1], [-4, 0]])
phase_portrait(A3, axes[2], 'Center')

plt.suptitle('Phase Portraits of 2D Linear Systems', fontsize=15, y=1.03)
plt.tight_layout()
plt.show()
print('Node: all trajectories converge. Saddle: unstable equilibrium.')
print('Center: closed orbits (conservative system -- like quantum HO).')

## 6. The Harmonic Oscillator: Exact vs Numerical

The **simple harmonic oscillator** (SHO) is described by

$$\ddot{x} + \omega^2 x = 0$$

with exact solution $x(t) = A\cos(\omega t + \phi)$.

This is the single most important ODE in physics:
- Classical mechanics: mass on a spring, pendulum
- Electromagnetism: LC circuits
- **Quantum mechanics:** the quantum harmonic oscillator is one of the few
  exactly solvable systems, with energy levels $E_n = \hbar\omega(n + \frac{1}{2})$.

We compare the **exact analytical solution** with a **numerical solution** from
`scipy.integrate.solve_ivp` and analyze the numerical error.

In [ ]:
# Harmonic oscillator: exact vs numerical solution
omega = 2.0  # Angular frequency
x0 = 1.0     # Initial displacement
v0 = 0.0     # Initial velocity

# Exact solution: x(t) = x0*cos(omega*t) + (v0/omega)*sin(omega*t)
def exact_solution(t):
    return x0 * np.cos(omega * t) + (v0 / omega) * np.sin(omega * t)

def exact_velocity(t):
    return -x0 * omega * np.sin(omega * t) + v0 * np.cos(omega * t)

# Numerical solution via solve_ivp (RK45)
def harmonic_rhs(t, state):
    x, v = state
    return [v, -omega**2 * x]

t_span = (0, 20)
t_eval = np.linspace(*t_span, 2000)

sol = solve_ivp(harmonic_rhs, t_span, [x0, v0], t_eval=t_eval,
                method='RK45', rtol=1e-12, atol=1e-14)

# Compute errors
x_exact = exact_solution(sol.t)
v_exact = exact_velocity(sol.t)
x_error = np.abs(sol.y[0] - x_exact)
v_error = np.abs(sol.y[1] - v_exact)

# Energy conservation check: E = 0.5*v^2 + 0.5*omega^2*x^2
energy_exact = 0.5 * v_exact**2 + 0.5 * omega**2 * x_exact**2
energy_num = 0.5 * sol.y[1]**2 + 0.5 * omega**2 * sol.y[0]**2

fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# (a) Position
axes[0, 0].plot(sol.t, x_exact, 'b-', lw=2, label='Exact')
axes[0, 0].plot(sol.t, sol.y[0], 'r--', lw=1, label='RK45')
axes[0, 0].set_xlabel('t')
axes[0, 0].set_ylabel('x(t)')
axes[0, 0].set_title('Position: Exact vs Numerical')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# (b) Phase space
axes[0, 1].plot(x_exact, v_exact, 'b-', lw=2, label='Exact (circle)')
axes[0, 1].plot(sol.y[0], sol.y[1], 'r--', lw=1, label='RK45')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('v = dx/dt')
axes[0, 1].set_title('Phase Space')
axes[0, 1].set_aspect('equal')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# (c) Error
axes[1, 0].semilogy(sol.t, x_error, 'b-', alpha=0.7, label='Position error')
axes[1, 0].semilogy(sol.t, v_error, 'r-', alpha=0.7, label='Velocity error')
axes[1, 0].set_xlabel('t')
axes[1, 0].set_ylabel('Absolute error')
axes[1, 0].set_title('Numerical Error (RK45)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, which='both')

# (d) Energy conservation
axes[1, 1].plot(sol.t, (energy_num - energy_num[0]) / energy_num[0], 'g-')
axes[1, 1].set_xlabel('t')
axes[1, 1].set_ylabel('Relative energy drift')
axes[1, 1].set_title('Energy Conservation')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle(f'Simple Harmonic Oscillator ($\\omega = {omega}$)', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

print(f'Max position error: {np.max(x_error):.2e}')
print(f'Max energy drift:   {np.max(np.abs(energy_num - energy_num[0]))/energy_num[0]:.2e}')

## 7. Quantum Mechanics Connection

### The Schrodinger Equation as a Differential Equation

The **time-independent Schrodinger equation** (TISE) is a second-order ODE:

$$-\frac{\hbar^2}{2m}\frac{d^2\psi}{dx^2} + V(x)\psi(x) = E\psi(x)$$

This is an **eigenvalue problem**: find the wavefunctions $\psi_n$ and
energies $E_n$ for a given potential $V(x)$.

### Classical to Quantum Harmonic Oscillator

The classical HO potential $V(x) = \frac{1}{2}m\omega^2 x^2$ leads to the
quantum energy levels:

$$E_n = \hbar\omega\left(n + \frac{1}{2}\right), \qquad n = 0, 1, 2, \ldots$$

The wavefunctions are **Hermite functions**:

$$\psi_n(x) = \frac{1}{\sqrt{2^n n!}}\left(\frac{m\omega}{\pi\hbar}\right)^{1/4}
e^{-m\omega x^2/(2\hbar)} H_n\!\left(\sqrt{\frac{m\omega}{\hbar}}\,x\right)$$

where $H_n$ are the Hermite polynomials.

In [ ]:
# Quantum Harmonic Oscillator wavefunctions and energy levels
from scipy.special import hermite
from math import factorial as fac

def qho_wavefunction(x, n, m=1.0, omega=1.0, hbar=1.0):
    '''Quantum harmonic oscillator wavefunction psi_n(x).'''
    xi = np.sqrt(m * omega / hbar) * x
    norm = (1.0 / np.sqrt(2**n * fac(n))) * (m * omega / (np.pi * hbar))**0.25
    Hn = hermite(n)
    return norm * np.exp(-xi**2 / 2) * Hn(xi)

def qho_energy(n, omega=1.0, hbar=1.0):
    '''Energy of the n-th level: E_n = hbar*omega*(n + 1/2).'''
    return hbar * omega * (n + 0.5)

# Plot first 5 wavefunctions and probability densities
x_vals = np.linspace(-5, 5, 500)
n_levels = 5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

colors = plt.cm.tab10(np.linspace(0, 0.5, n_levels))

for n in range(n_levels):
    psi = qho_wavefunction(x_vals, n)
    E_n = qho_energy(n)

    # Offset by energy level for visualization
    ax1.plot(x_vals, psi + E_n, color=colors[n], lw=1.5,
             label=rf'$\psi_{n}$, $E_{n}={E_n:.1f}\,\hbar\omega$')
    ax1.axhline(E_n, color=colors[n], ls='--', alpha=0.3)

    ax2.plot(x_vals, np.abs(psi)**2 + E_n, color=colors[n], lw=1.5,
             label=rf'$|\psi_{n}|^2$')
    ax2.axhline(E_n, color=colors[n], ls='--', alpha=0.3)

# Add the potential
V = 0.5 * x_vals**2
ax1.plot(x_vals, V, 'k-', lw=2, alpha=0.3, label=r'$V(x) = \frac{1}{2}\omega^2 x^2$')
ax2.plot(x_vals, V, 'k-', lw=2, alpha=0.3, label=r'$V(x)$')

ax1.set_xlabel('x')
ax1.set_ylabel(r'$\psi_n(x) + E_n$')
ax1.set_title('Wavefunctions (offset by energy)')
ax1.legend(fontsize=8, loc='upper right')
ax1.set_ylim(-0.5, 6)
ax1.grid(True, alpha=0.2)

ax2.set_xlabel('x')
ax2.set_ylabel(r'$|\psi_n(x)|^2 + E_n$')
ax2.set_title('Probability Densities (offset by energy)')
ax2.legend(fontsize=8, loc='upper right')
ax2.set_ylim(-0.5, 6)
ax2.grid(True, alpha=0.2)

plt.suptitle('Quantum Harmonic Oscillator', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

# Verify orthonormality
print('Orthonormality check: <psi_m | psi_n> (should be delta_mn)')
print('-' * 50)
for m in range(4):
    for n in range(m, 4):
        integrand = qho_wavefunction(x_vals, m) * qho_wavefunction(x_vals, n)
        inner_product = np.trapezoid(integrand, x_vals)
        expected = 1.0 if m == n else 0.0
        print(f'  <{m}|{n}> = {inner_product:8.5f}  (expect {expected:.1f})')

In [ ]:
# Numerical solution of the Schrodinger equation for the QHO
# Using the finite-difference matrix diagonalization approach

# Discretize the TISE: -d^2 psi/dx^2 + x^2 psi = 2E psi  (natural units)
# Using finite differences: d^2psi/dx^2 ~ (psi_{i+1} - 2*psi_i + psi_{i-1})/dx^2

N = 500          # Grid points
x_max = 8.0
x_grid = np.linspace(-x_max, x_max, N)
dx = x_grid[1] - x_grid[0]

# Construct the Hamiltonian matrix (tridiagonal + potential)
# H = -d^2/dx^2 + V(x), using hbar = m = omega = 1
# Kinetic energy: T_{ij} for tridiagonal
diag = 1.0 / dx**2 + 0.5 * x_grid**2  # Main diagonal: kinetic + potential
off_diag = -0.5 / dx**2 * np.ones(N - 1)  # Off-diagonal: kinetic

# Build sparse tridiagonal matrix
H = np.diag(diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1)

# Solve eigenvalue problem
eigenvalues, eigenvectors = np.linalg.eigh(H)

# Compare first several eigenvalues with exact: E_n = n + 0.5
n_compare = 8
print('Numerical vs Exact Energy Levels (hbar = m = omega = 1)')
print('=' * 55)
print(f'{"n":>3s} {"Numerical":>14s} {"Exact":>14s} {"Error":>12s}')
print('-' * 55)
for n in range(n_compare):
    E_exact = n + 0.5
    E_num = eigenvalues[n]
    error = abs(E_num - E_exact)
    print(f'{n:3d} {E_num:14.8f} {E_exact:14.8f} {error:12.2e}')

# Plot numerical vs analytical wavefunctions
fig, axes = plt.subplots(2, 2, figsize=(13, 9))
for idx, ax in enumerate(axes.flat):
    n = idx
    # Numerical wavefunction (normalize)
    psi_num = eigenvectors[:, n]
    psi_num = psi_num / np.sqrt(np.trapezoid(psi_num**2, x_grid))
    # Fix sign ambiguity
    if psi_num[N//2] < 0 and n % 2 == 0:
        psi_num = -psi_num
    if psi_num[N//2 + 10] < 0 and n % 2 == 1:
        psi_num = -psi_num

    # Analytical
    psi_exact = qho_wavefunction(x_grid, n)

    ax.plot(x_grid, psi_exact, 'b-', lw=2, label='Analytical')
    ax.plot(x_grid, psi_num, 'r--', lw=1.5, label='Numerical')
    ax.set_xlabel('x')
    ax.set_ylabel(rf'$\psi_{n}(x)$')
    ax.set_title(rf'$n = {n}$, $E_{n} = {eigenvalues[n]:.4f}$ '
                 rf'(exact: {n+0.5:.1f})')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-5, 5)

plt.suptitle('Quantum HO: Numerical (finite diff.) vs Analytical',
             fontsize=15, y=1.02)
plt.tight_layout()
plt.show()
print('The finite-difference method recovers the exact QHO solutions')
print('with high accuracy -- a preview of computational quantum mechanics.')

## Summary

### Key Formulas

| Concept | Formula |
|---|---|
| Partial derivative | $\frac{\partial f}{\partial x} = \lim_{h\to 0}\frac{f(x+h,y)-f(x,y)}{h}$ |
| Gradient | $\nabla f = (\partial_x f,\, \partial_y f)$ |
| Green's theorem | $\oint_C(Pdx+Qdy) = \iint_D(Q_x - P_y)\,dA$ |
| SHO solution | $x(t) = A\cos(\omega t + \phi)$ |
| Phase portrait | $\dot{\mathbf{x}} = A\mathbf{x}$, classified by eigenvalues of $A$ |
| TISE | $-\frac{\hbar^2}{2m}\psi'' + V\psi = E\psi$ |
| QHO energies | $E_n = \hbar\omega(n + \frac{1}{2})$ |

### Main Takeaways

1. Multivariable calculus extends derivatives and integrals to higher dimensions -- essential for 3D quantum systems.
2. The gradient is the key to optimization and appears in quantum mechanics as the momentum operator (up to constants).
3. Green's theorem connects line integrals to area integrals; it generalizes to Stokes' theorem used in gauge theory.
4. ODEs are solved analytically (SymPy `dsolve`) and numerically (`scipy.integrate.solve_ivp`); both approaches are used in QM.
5. Phase portraits reveal the qualitative behavior of dynamical systems -- centers correspond to quantum bound states.
6. The harmonic oscillator bridges classical and quantum physics: the classical ODE becomes the quantum eigenvalue problem.
7. Numerical methods (finite differences) can solve the Schrodinger equation, recovering exact results for exactly solvable potentials.

---
*Next: Linear Algebra I (Month 4) -- the language of quantum mechanics.*